# DF

In [7]:
%store -r df_one_hot

# PACKAGES

In [8]:
# package that we need
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import KFold

# essential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#correlation
import seaborn as sns

#! pip install pydot

# FEATURE AND TARGET

In [9]:
numerical_features = [
    'spotify_track_duration_minute', 'danceability', 'energy', 'loudness',
    'mode', 'speechiness', 'acousticness',
    'spotify_track_popularity', 'instrumentalness', 'liveness']

genre_features = [col for col in df_one_hot.columns if col.startswith('genre_')]

# UNION OF THE TWO 
total_features = numerical_features + genre_features

target='valence'

# PLOT

interesting plots for visualize the output of our model

In [10]:
def plot_for_model(history):

    #plot loss
    plt.figure()
    plt.plot(history.history['loss'],color='blue',label='loss')
    plt.plot(history.history['val_loss'],color='red',label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss function through epoch')
    plt.legend()
    plt.grid()
    plt.show()

    #plot mean
    plt.figure()
    plt.plot(history.history['mae'],color='blue',label='mae')
    plt.plot(history.history['val_mae'],color='red',label='val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('mae')
    plt.title('Mae function through epoch')
    plt.grid()
    plt.legend()
    plt.show()   

# SPLIT DATA

In [11]:
def return_train_test_for_cross(train_index, test_index, df):
    #extract the feature and target
    x = df[total_features]
    y = df[target]

    #normalization
    scaler = StandardScaler()
    x[numerical_features] = scaler.fit_transform(x[numerical_features])
    #display(x)

    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #display(x_train)

    return x_train,x_test,y_train,y_test

# MODEL

In [12]:
# function to create the model
def create_model_user(shape_x_train): #number of columns -> shape
    
    
    '''model_user_i = Sequential([
        Dense(32, input_dim=shape_x_train, activation='relu', name='first_layer'),
        Dense(16, activation='relu', name='secod_layer'),
        Dense(1, activation='sigmoid', name='final_layer')
    ])'''

    model_user_i = Sequential([
        
        Dense(32,input_dim=shape_x_train,activation='relu',name='first_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(16,activation='relu',name='secod_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        #Dense(1,activation='linear',name='final_layer')
        Dense(1,activation='sigmoid',name='final_layer') 
    ])

    #compile the model
    model_user_i.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    #summary
    #display(model_user_i.summary())

    #return  the model
    return model_user_i
    

# FIT MODEL

In [13]:
#function to train the model
def fit_model(model, x_train, y_train, x_test, y_test):

    history_model = model.fit(
        x_train,y_train,
        validation_data = [x_test, y_test],
        #epochs = 1,
        epochs=100,
        batch_size=32,
        #verbose=1
    )

    return history_model

# RECOMANDATION FUNCTION + CONFUSION MATRIX

In [14]:
def reco_function_conf_matrix(y_test, y_hat, threshold):
    
    #initialize dataframe    
    data = {'Original Test': y_test,
        'Prediction Test': y_hat.flatten(),
        }
    df_prediction = pd.DataFrame(data = data)
    df_prediction['Original over Thrashold'] = df_prediction['Original Test'] > threshold
    df_prediction['Predicted over Thrashold'] = df_prediction['Prediction Test'] > threshold
    #display(df_prediction)

    #plot confusion matrix of reccomandation
    cm = confusion_matrix(df_prediction['Original over Thrashold'], df_prediction['Predicted over Thrashold'])

    # METRIX FOR EVALUATE RECCOMANDATION 
    # Extract TP, TN, FP, FN
    TN, FP, FN, TP = cm.ravel()

    # Calculate metrics manually
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    '''plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['True', 'False'], yticklabels=['True', 'False'])
    plt.title('Confiusion matrix')
    plt.xlabel('predicted')
    plt.ylabel('actual')
    plt.show()'''

    # Store metrics in a DataFrame
    df_metrics_cm = pd.DataFrame(
        data =
        #'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
        {'Value': [accuracy, precision, recall, f1]},
        index = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    )

    #display(df_metrics_cm.round(2))

    return df_prediction,df_metrics_cm

# MAIN 

In [15]:
dizio_model_user = {} #in this dizio we store the model for each user
dizio_accuracy = {} #accuracy of the model

# DICTIONARY TO STORE SONG RECOMMENDATION PER USER
reccomandations_songs_per_user={}

#threshold
threshold = 0.7

#value k for k-fold
k = 5

#loop creation user and model
for i in range(0, len(df_one_hot['User'].unique())): #i indica lo user   

    print('---------------------------------------------------------')
    print(f'USER {i}')

    #def dizio
    df_metrics_kfold = pd.DataFrame()

    #select user
    mask_user = df_one_hot['User'] == i
    #display(mask_user)
    user_i_df = df_one_hot[mask_user]
    #display(user_i_df) 

    #cross-validation k-fold
    kf = KFold(n_splits = k, shuffle= True, random_state= 42)
    score_fold = []

    #loop on the k-th folder
    loop = 0
    for train_index, test_index in kf.split(user_i_df):
        
        print('---------------------------------------------------------')
        print(f'K {loop} FOLD') 
        loop = loop + 1

        #return train and test
        x_train,x_test,y_train,y_test = return_train_test_for_cross(train_index, test_index, user_i_df) #da modificare

        #creation model for the specific user
        model_user_i = create_model_user(x_train.shape[1])
        
        #fit the model with the different x, y train (for different user)
        history_model_user_i = fit_model(model_user_i, x_train, y_train, x_test, y_test)
        
        #prediction
        y_hat = model_user_i.predict(x_test)

        #plot the metrics of the model
        #plot_for_model(history_model_user_i)

        #accuracy
        test_loss, test_mae = model_user_i.evaluate(x_test,y_test,verbose=1)
        #print('MSE: ', test_loss)
        #print('MAE:', test_mae)

        score_fold.append(test_mae)

        #reccomandation
        reccomandation_df = reco_function_conf_matrix(y_test, y_hat, threshold)
        df_metrics_kfold = pd.concat([df_metrics_kfold,reccomandation_df[1]], axis = 1)

    #save the model in the dictionary
    #add also the raccomandations
    dizio_model_user[f"User{i}"] = {#'model_user_i' : model_user_i,
                                    #'history_model_user_i': history_model_user_i,
                                    #'x_train': x_train,
                                    #'x_test': x_test,
                                    #'y_train': y_train,
                                    #'y_test': y_test,
                                    #'y_hat': y_hat,
                                    #'recommendations:': reccomandation_df
                                    'score_fold': score_fold,
                                    'df_metrics_kfold_mean': pd.DataFrame(data = df_metrics_kfold.mean(axis=1), columns=['Mean_Metrics'])
                                    } 
    #save accuracy 
    dizio_accuracy[f'User{i}'] = {
        #'MAE' : mean_absolute_error(y_test, y_hat),
        #'MSE' : mean_squared_error(y_test, y_hat)
        'Mean MAE': np.mean(score_fold)
    }

---------------------------------------------------------
USER 0
---------------------------------------------------------
K 0 FOLD


C:\Users\lovoa\AppData\Local\Temp\ipykernel_1432\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.4517 - mae: 0.1916 - val_loss: 0.1910 - val_mae: 0.1662
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1555 - mae: 0.1717 - val_loss: 0.0770 - val_mae: 0.1598
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0691 - mae: 0.1627 - val_loss: 0.0467 - val_mae: 0.1501
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0464 - mae: 0.1548 - val_loss: 0.0399 - val_mae: 0.1468
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0418 - mae: 0.1530 - val_loss: 0.0381 - val_mae: 0.1453
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0402 - mae: 0.1487 - val_loss: 0.0374 - val_mae: 0.1439
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0396 - mae: 0.1494 - val_loss: 0.0370 - val_mae: 0.1443
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0395 - mae: 0.1490 - val_loss: 0.0364 - val_mae: 0.1420
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

C:\Users\lovoa\AppData\Local\Temp\ipykernel_1432\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4434 - mae: 0.1905 - val_loss: 0.1925 - val_mae: 0.1672
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1550 - mae: 0.1669 - val_loss: 0.0785 - val_mae: 0.1584
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0675 - mae: 0.1568 - val_loss: 0.0488 - val_mae: 0.1536
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0465 - mae: 0.1541 - val_loss: 0.0414 - val_mae: 0.1503
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0418 - mae: 0.1527 - val_loss: 0.0399 - val_mae: 0.1505
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0395 - mae: 0.1494 - val_loss: 0.0392 - val_mae: 0.1494
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0401 - mae: 0.1519 - val_loss: 0.0387 - val_mae: 0.1473
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0385 - mae: 0.1477 - val_loss: 0.0384 - val_mae: 0.1480
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5387 - mae: 0.2316 - val_loss: 0.3624 - val_mae: 0.1932
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3297 - mae: 0.2048 - val_loss: 0.2251 - val_mae: 0.1832
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2057 - mae: 0.1909 - val_loss: 0.1443 - val_mae: 0.1782
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1342 - mae: 0.1850 - val_loss: 0.0977 - val_mae: 0.1723
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0942 - mae: 0.1814 - val_loss: 0.0715 - val_mae: 0.1659
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0699 - mae: 0.1727 - val_loss: 0.0568 - val_mae: 0.1599
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0564 - mae: 0.1636 - val_loss: 0.0492 - val_mae: 0.1561
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0488 - mae: 0.1588 - val_loss: 0.0449 - val_mae: 0.1531
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0465 - mae: 0.1

C:\Users\lovoa\AppData\Local\Temp\ipykernel_1432\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5207 - mae: 0.2240 - val_loss: 0.3536 - val_mae: 0.1899
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3167 - mae: 0.1883 - val_loss: 0.2211 - val_mae: 0.1796
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1994 - mae: 0.1795 - val_loss: 0.1434 - val_mae: 0.1747
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1305 - mae: 0.1724 - val_loss: 0.0980 - val_mae: 0.1694
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0905 - mae: 0.1673 - val_loss: 0.0720 - val_mae: 0.1642
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0701 - mae: 0.1684 - val_loss: 0.0572 - val_mae: 0.1599
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0543 - mae: 0.1561 - val_loss: 0.0493 - val_mae: 0.1558
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0488 - mae: 0.1536 - val_loss: 0.0447 - val_mae: 0.1533
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0444 - mae: 0.1

C:\Users\lovoa\AppData\Local\Temp\ipykernel_1432\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5105 - mae: 0.2102 - val_loss: 0.3437 - val_mae: 0.1808
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3082 - mae: 0.1783 - val_loss: 0.2120 - val_mae: 0.1741
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1942 - mae: 0.1810 - val_loss: 0.1347 - val_mae: 0.1690
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1240 - mae: 0.1710 - val_loss: 0.0907 - val_mae: 0.1655
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0865 - mae: 0.1692 - val_loss: 0.0657 - val_mae: 0.1597
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0649 - mae: 0.1620 - val_loss: 0.0523 - val_mae: 0.1541
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0532 - mae: 0.1557 - val_loss: 0.0452 - val_mae: 0.1521
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0484 - mae: 0.1582 - val_loss: 0.0417 - val_mae: 0.1498
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0445 - mae: 0.1

# DISPLAY DICTIONARY ACCURAY

In [16]:
display(dizio_accuracy)

{'User0': {'Mean MAE': 0.1376309871673584},
 'User1': {'Mean MAE': 0.1329177975654602},
 'User2': {'Mean MAE': 0.09580752849578858},
 'User3': {'Mean MAE': 0.1331585168838501},
 'User4': {'Mean MAE': 0.1377421259880066}}

# DISPLAY DF METRICS CONFUSION MATRIX

In [17]:
dizio_model_user['User4']['df_metrics_kfold_mean']

,Mean_Metrics
Accuracy,0.773801
Precision,0.774979
Recall,0.549971
F1-score,0.642376


In [18]:
df_metrics_total = pd.concat([dizio_model_user['User0']['df_metrics_kfold_mean'],dizio_model_user['User1']['df_metrics_kfold_mean'],dizio_model_user['User2']['df_metrics_kfold_mean'],dizio_model_user['User3']['df_metrics_kfold_mean'],dizio_model_user['User4']['df_metrics_kfold_mean']], axis=1)
#type(df_metrics_total)
#df_metrics_total.columns(colume_name = ['User0','User1','User2','User3','User4'])
df_metrics_total.columns = ['User0','User1','User2','User3','User4']
display(df_metrics_total)

,User0,User1,User2,User3,User4
Accuracy,0.739273,0.804883,0.823721,0.822511,0.773801
Precision,0.728627,0.785661,0.749835,0.808589,0.774979
Recall,0.537818,0.676216,0.715534,0.339071,0.549971
F1-score,0.613859,0.726151,0.729466,0.475305,0.642376


In [19]:
df_metrics_mean = pd.DataFrame(data = df_metrics_total.mean(axis=1), columns=['Mean_Metrics'])
display(df_metrics_mean.round(3))

,Mean_Metrics
Accuracy,0.793
Precision,0.770
Recall,0.564
F1-score,0.637
